In [1]:
import requests
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go

firstUserID = 579
# lastUserID = 973
lastUserID = 1887

g_isLevel8 = False
g_isLevel8Ex = False
g_isLevel9 = True

g_isRegDay = False
g_regDay = '2021-10-15'

g_fnHead = ''

if g_isLevel8:
    g_fnHead = 'l8.'
    
if g_isLevel8Ex:
    g_fnHead = 'l8ex.'    
    
if g_isLevel9:
    g_fnHead = 'l9.'    

if g_isRegDay:
    crday = datetime.strptime(g_regDay, '%Y-%m-%d')
    g_fnHead = '{}{}{}.'.format(crday.year - 2000, crday.month, crday.day)

pd.set_option("display.max_rows", 1000)

In [2]:
# 得到用户统计数据
def getUserStats(uid):
    r = requests.get('https://block7serv.heyalgo.io/v1/games/userstats?token=ewfbneivneghy3uryh&uid={}'.format(uid))
    return r.json()

# 解析时间字符串，2021-10-13_10:01:48
def parseTime(str):
    if str == '':
        return None
    
    return datetime.strptime(str, '%Y-%m-%d_%H:%M:%S')

# 计算时间差，返回整数小时差
def getTimeOffsetHours(timeNow, timeStart):
    if timeStart == None:
        return -1
    
    return int((timeNow.timestamp() - timeStart.timestamp()) / 60 / 60)


# form timestamp
def fromTimestamp(ts):
    return datetime.fromtimestamp(ts)

# dt = parseTime('2021-10-13_10:01:48')
# datetime.now()
# getTimeOffsetHours(datetime.now(), dt)

In [3]:
# 计算平均关卡星星，需要跳过5的倍数的关卡
def countAvgUserStageStars(levelArr):
    if levelArr == None:
        return 0
    
    totalstars = 0
    stagenums = 0
    
    for key in levelArr.keys():
        if int(key) % 5 != 0:
            totalstars = totalstars + int(levelArr[key])
            stagenums = stagenums + 1
    
    if stagenums > 0:
        return totalstars / stagenums
    
    return 0

# 计算平均关卡星星，需要跳过5的倍数的关卡
def analyzeAvgStageStars(lststages, levelArr):
    if levelArr == None:
        return 
    
    for key in levelArr.keys():
        if int(key) % 5 != 0:
            addStageStar(lststages, int(key), int(levelArr[key]))

            
# 计算下一关的流失率
def countNextStage(lststages, stage, curnums):
    if curnums > 0:
        for s in lststages:
            if s['stage'] == stage + 1:
                s['lostper'] = (curnums - s['totalusers']) / curnums
                
                if s['lostper'] < 0:
                    s['lostper'] = 0

                return
    else:
        s['lostper'] = 0
        
# 获取前一关的游戏人数       
def getPreStageTotalUsers(lststages, stage):
    for s in lststages:
        if s['stage'] == stage - 1:
            return s['totalusers']
            
    return 0

# new stage stats data
def newStageData(stage):
    ss = {
        'stage': stage,
        'totalwinper': 0,
        'totalusers': 0,
        'winper': 0,
        'totalClickNums': 0,
        'avgClickNums': 0,
        'totalAvgClickTime': 0,
        'avgClickTime': 0,
        'totalnums': 0,
        'totalWinClickNums': 0,
        'avgWinClickNums': 0,
        'totalWinAvgClickTime': 0,
        'avgWinClickTime': 0,
        'totalWinNums': 0,
        'totalLoseClickNums': 0,
        'avgLoseClickNums': 0,
        'totalLoseAvgClickTime': 0,
        'avgLoseClickTime': 0,
        'totalLoseNums': 0,
        'stayUsers': 0,
        'stayUsersPer': 0,    
        'avgLoseProgress': 0,
        'totalGameNums': 0,
        'totalGameWinNums': 0,
        'gameWinPer': 0,
        'totalStars': 0,
        'totalStarUserNums': 0,
        'avgStars': 0,
    }
    
    return ss


# 新增加一个玩家关卡统计
def addStageStar(lststages, stage, stars):
    for s in lststages:
        if s['stage'] == stage:
            s['totalStars'] = s['totalStars'] + stars
            s['totalStarUserNums'] = s['totalStarUserNums'] + 1
            s['avgStars'] = s['totalStars'] / s['totalStarUserNums']
            
            return
    
    ss = newStageData(stage)
    
    ss['totalStars'] = ss['totalStars'] + stars
    ss['totalStarUserNums'] = ss['totalStarUserNums'] + 1
    ss['avgStars'] = ss['totalStars'] / ss['totalStarUserNums']
    
    lststages.append(ss)


# 新增加一个玩家关卡统计
def addUserStages(lststages, stage, winper, winnums, totalnums):
    for s in lststages:
        if s['stage'] == stage:
            s['totalGameWinNums'] = s['totalGameWinNums'] + winnums
            s['totalGameNums'] = s['totalGameNums'] + totalnums
            if s['totalGameNums'] > 0:
                s['gameWinPer'] = s['totalGameWinNums'] / s['totalGameNums']
            
            s['totalwinper'] = s['totalwinper'] + winper
            s['totalusers'] = s['totalusers'] + 1
            s['winper'] = s['totalwinper'] / s['totalusers']
            
            ptnums = getPreStageTotalUsers(lststages, stage)
            if ptnums != 0:
                s['lostper'] = (ptnums - s['totalusers'])  / ptnums
                
                if s['lostper'] < 0:
                    s['lostper'] = 0
            else:
                s['lostper'] = 0
            
            countNextStage(lststages, stage, s['totalusers'])
            
            return
    
    ss = newStageData(stage)
    
    ss['totalwinper'] = winper
    ss['totalusers'] = 1
    ss['winper'] = winper
    
    ss['totalGameWinNums'] = ss['totalGameWinNums'] + winnums
    ss['totalGameNums'] = ss['totalGameNums'] + totalnums
    if ss['totalGameNums'] > 0:
        ss['gameWinPer'] = ss['totalGameWinNums'] / ss['totalGameNums']    
    
    lststages.append(ss)
    
    ss['lostper'] = (getPreStageTotalUsers(lststages, stage) - ss['totalusers']) / ss['totalusers']        
    countNextStage(lststages, stage, ss['totalusers'])
    
    
def procUserStayStage(lststages, us):
    for s in lststages:
        if s['stage'] == int(us['user']['level']):
            s['stayUsers'] = s['stayUsers'] + 1
            
            if s['totalusers'] > 0:
                s['stayUsersPer'] = s['stayUsers'] / s['totalusers']
            
            return


# 分析玩家历史数据
def analyzeUserStageHistory(ui, lststages, stage, stageHistory):
    for s in lststages:
        if s['stage'] == stage:
            s['totalnums'] = s['totalnums'] + 1
            
            s['totalClickNums'] = s['totalClickNums'] + stageHistory['clickNums']
            s['avgClickNums'] = s['totalClickNums'] / s['totalnums'] 
            
            s['totalAvgClickTime'] = s['totalAvgClickTime'] + stageHistory['avgClickTime']
            s['avgClickTime'] = s['totalAvgClickTime'] / s['totalnums']
            
            ui['totalHistoryNums'] = ui['totalHistoryNums'] + 1
            ui['totalAvgClickTime'] = ui['totalAvgClickTime'] + stageHistory['avgClickTime']
            ui['avgClickTime'] = ui['totalAvgClickTime'] / ui['totalHistoryNums']
            
            if stageHistory['gamestate'] == 1:
                s['totalWinNums'] = s['totalWinNums'] + 1

                s['totalWinClickNums'] = s['totalWinClickNums'] + stageHistory['clickNums']
                s['avgWinClickNums'] = s['totalWinClickNums'] / s['totalWinNums'] 

                s['totalWinAvgClickTime'] = s['totalWinAvgClickTime'] + stageHistory['avgClickTime']
                s['avgWinClickTime'] = s['totalWinAvgClickTime'] / s['totalWinNums']                
            else:
                s['totalLoseNums'] = s['totalLoseNums'] + 1

                s['totalLoseClickNums'] = s['totalLoseClickNums'] + stageHistory['clickNums']
                s['avgLoseClickNums'] = s['totalLoseClickNums'] / s['totalLoseNums'] 

                s['totalLoseAvgClickTime'] = s['totalLoseAvgClickTime'] + stageHistory['avgClickTime']
                s['avgLoseClickTime'] = s['totalLoseAvgClickTime'] / s['totalLoseNums']
                
            if s['avgWinClickNums'] > 0:
                s['avgLoseProgress'] = s['avgLoseClickNums'] / s['avgWinClickNums']
                
            return
            
    
# 分析玩家关卡数据    
def analyzeUserStages(ui, lststages, lstdaystats, stages):
    if stages == None:
        ui['stagenums'] = 0
        
        return 
    
    nums = 0
    for key in stages.keys():
        if len(stages[key]['historys']) > 0:
            addUserStages(lststages, int(key), stages[key]['winnums'] / len(stages[key]['historys']), 
                          stages[key]['winnums'], len(stages[key]['historys']))
            nums = nums + len(stages[key]['historys'])
            
            for sh in stages[key]['historys']:
                analyzeUserStageHistory(ui, lststages, int(key), sh)
                onUserStage(lstdaystats, ui['uid'], ui['createTime'], fromTimestamp(sh['ts']))
                
            
    ui['stagenums'] = nums
        
    return

# 是否是一个有效的用户
def isValidUserStatsData(usret):
    if usret == None:
        return False
    
    if usret['user'] == None:
        return False
    
    if g_isLevel8:
        if usret['user']['level'] != 8:
            return False
        
    if g_isLevel8Ex:
        if usret['user']['level'] != 8 and usret['user']['level'] != 9:
            return False
        
    if g_isLevel9:
        if usret['user']['level'] != 9:
            return False        
        
    if g_isRegDay:
        if usret['user']['createTime'].find(g_regDay) != 0:
            return False
    
    if usret['user']['stages'] == None:
        return False
    
    for key in usret['user']['stages'].keys():
        return True
    
    return False

def newDayStats(dt):
    ds = {
        'date': dt,
        'title': '{}-{}-{}'.format(dt.year, dt.month, dt.day),
        'uids': [],
        'newusers': 0,
        'dates': [],
        'days': [],
        'aliveusers': [],
        'aliveuids': [],        
    }
    
    return ds

# 新增一天
def addDayInDayStats(lstdaystats, uid, dt):
    for ds in lstdaystats:
        if ds['date'].year == dt.year and ds['date'].month == dt.month and ds['date'].day == dt.day:
            ds['uids'].append(uid)
            ds['newusers'] = len(ds['uids'])
            
            return 
    
    ds = newDayStats(dt)
    
    ds['uids'].append(uid)
    ds['newusers'] = len(ds['uids'])
    
    lstdaystats.append(ds)
    

def onUserStageDS(ds, uid, gamedt):
    for i in range(len(ds['days'])):
        if ds['dates'][i].year == gamedt.year and ds['dates'][i].month == gamedt.month and ds['dates'][i].day == gamedt.day:
            for cuid in ds['aliveuids'][i]:
                if cuid == uid:
                    return
            
            ds['aliveuids'][i].append(uid)
            ds['aliveusers'][i] = len(ds['aliveuids'][i])
            
            return 
        
    ds['dates'].append(gamedt)
    ds['days'].append('{}-{}-{}'.format(gamedt.year, gamedt.month, gamedt.day))
    ds['aliveuids'].append([uid])
    ds['aliveusers'].append(1)   
    
    return
    
def onUserStage(lstdaystats, uid, newdt, gamedt):
    if newdt.timestamp() < gamedt.timestamp():
        for ds in lstdaystats:
            if ds['date'].year == newdt.year and ds['date'].month == newdt.month and ds['date'].day == newdt.day:
                
                onUserStageDS(ds, uid, gamedt)
                
                return
    

# 分析用户数据
def analyzeUserStats(startUID, endUID):
    lstui = []
    lststages = []
    lstdaystats = []
    timeNow = datetime.now()
    
    for uid in range(startUID, endUID + 1):
        cui = getUserStats(uid)
        if isValidUserStatsData(cui):
            ui = {
                'uid': uid, 
                'coin': int(cui['user']['coin']),
                'level': int(cui['user']['level']),
                'createTime': parseTime(cui['user']['createTime']),
                'lastTime': parseTime(cui['user']['lastLoginTime']),
                'avgStars': countAvgUserStageStars(cui['user']['levelarr']),
                'ip': cui['user']['ip'],
                'totalAvgClickTime': 0,
                'avgClickTime': 0,
                'totalHistoryNums': 0,
            }
            
            addDayInDayStats(lstdaystats, uid, ui['createTime'])

            if ui['lastTime'] == None:
                ui['lastTime'] = ui['createTime']

            ui['offlineHours'] = getTimeOffsetHours(timeNow, ui['lastTime'])
            ui['aliveHours'] = getTimeOffsetHours(ui['lastTime'], ui['createTime'])

            analyzeUserStages(ui, lststages, lstdaystats, cui['user']['stages'])

            procUserStayStage(lststages, cui)
            
            analyzeAvgStageStars(lststages, cui['user']['levelarr'])

            lstui.append(ui)
    
    return lstui, lststages, lstdaystats

In [ ]:
lstui, lststages, lstdaystats = analyzeUserStats(firstUserID, lastUserID)

In [ ]:
df = pd.DataFrame(lstui)
df['createTime'] = pd.to_datetime(df['createTime'])

# df[(df['aliveHours'] > 0)&(df['level'] > 0)].sort_values("level")

In [ ]:
# df[(df['createTime'] > datetime.strptime('2021-10-13', '%Y-%m-%d'))&(df['createTime'] < datetime.strptime('2021-10-14', '%Y-%m-%d'))]

In [ ]:
df1 = pd.DataFrame(lststages)

# df1.sort_values("stage")

In [ ]:
def showStagesStats(df, fn):
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['totalusers'],
        name='参与用户数',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['winper'],
        name='平均用户通关率',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['totalnums'],
        name='挑战次数',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['avgClickTime'],
        name='平均点击时间差',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['avgWinClickTime'],
        name='平均胜利点击时间差',
    ))    
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['avgLoseClickTime'],
        name='平均失败点击时间差',
    ))    
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['lostper'],
        name='流失率',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['stayUsers'],
        name='滞留用户数',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['stayUsersPer'],
        name='滞留用户比例',
    ))    
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['avgLoseProgress'],
        name='平均失败进度',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['gameWinPer'],
        name='通关率',
    ))    
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['avgStars'],
        name='平均星星数',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['totalStarUserNums'],
        name='获得星星玩家数',
    ))    

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.write_html('{}{}'.format(g_fnHead, fn))
    fig.show()
    
showStagesStats(df1, 'stagestats.html')

In [ ]:
def showDayStats(lstdaystats):
    fig = go.Figure()
    
    for d in lstdaystats:
        lsti = []
        for i in range(len(d['days'])):
            lsti.append(
                (datetime.strptime(d['days'][i], '%Y-%m-%d') - datetime.strptime(d['title'], '%Y-%m-%d')).days)
            
        fig.add_trace(go.Bar(
            x=lsti,
            y=d['aliveusers'],
            name=d['title'],
        ))
        
        lst = []
        for v in d['aliveusers']:
            lst.append(v / d['newusers'])
            
        fig.add_trace(go.Bar(
            x=lsti,
            y=lst,
            name='{}留存率'.format(d['title']),
        ))            

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.write_html('{}{}'.format(g_fnHead, "daystats.html"))
    fig.show()
    
showDayStats(lstdaystats)

In [ ]:
def countNums(sarr, offv):
    arr1 = []
    arr0 = []
    n = sarr.max() / offv
    for t in range(int(n) + 1):
        arr0.append(t * offv)
        arr1.append(0)
        
    for v in sarr:
        arr1[int(v / offv)] = arr1[int(v / offv)] + 1
    
    return arr0, arr1

In [ ]:
def showUserStats(df, fn):
    fig = go.Figure()
    
    arr0, arr1 = countNums(df['level'], 1)
    fig.add_trace(go.Bar(
            x=arr0,
            y=arr1,
            name='关卡统计',
        ))
    
    arr0, arr1 = countNums(df['level'], 5)
    fig.add_trace(go.Bar(
            x=arr0,
            y=arr1,
            name='关卡统计（步长5）',
        ))    

    arr0, arr1 = countNums(df['coin'], 10)
    fig.add_trace(go.Bar(
            x=arr0,
            y=arr1,
            name='剩余金币统计（步长10）',
        ))    
    
    arr0, arr1 = countNums(df['coin'], 50)
    fig.add_trace(go.Bar(
            x=arr0,
            y=arr1,
            name='剩余金币统计（步长50）',
        ))    
    
    arr0, arr1 = countNums(df['avgClickTime'], 10)
    fig.add_trace(go.Bar(
            x=arr0,
            y=arr1,
            name='点击时间差统计（步长10）',
        ))
    
    arr0, arr1 = countNums(df['aliveHours'], 2)
    fig.add_trace(go.Bar(
            x=arr0,
            y=arr1,
            name='活跃时长（小时）（步长2）',
        ))
    
    arr0, arr1 = countNums(df['offlineHours'], 12)
    fig.add_trace(go.Bar(
            x=arr0,
            y=arr1,
            name='离线时长（小时）（步长12）',
        ))    
    
    arr0, arr1 = countNums(df['totalHistoryNums'], 10)
    fig.add_trace(go.Bar(
            x=arr0,
            y=arr1,
            name='挑战次数（步长10）',
        ))        

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.write_html('{}{}'.format(g_fnHead, fn))
    fig.show()
    
showUserStats(df, 'userstats.html')

In [ ]:
def getUserDFWithDay(df, sd, ed):
    return df[(df['createTime'] > datetime.strptime(sd, '%Y-%m-%d'))&(df['createTime'] < datetime.strptime(ed, '%Y-%m-%d'))]

if not g_isRegDay:
    showUserStats(getUserDFWithDay(df, '2021-10-12', '2021-10-13'), 'userstats211012.html')
    showUserStats(getUserDFWithDay(df, '2021-10-13', '2021-10-14'), 'userstats211013.html')
    showUserStats(getUserDFWithDay(df, '2021-10-14', '2021-10-15'), 'userstats211014.html')
    showUserStats(getUserDFWithDay(df, '2021-10-15', '2021-10-16'), 'userstats211015.html')